In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
from datetime import datetime
import numpy as np
import requests
requests.packages.urllib3.disable_warnings()
from bs4 import BeautifulSoup
import time #Sleep between downloads

In [33]:
# Read files and subset the dataset
refs = pd.read_csv("Source database.csv", encoding = "utf-8")
refs_nct = refs[(refs.database == "yes") & (refs.search == "clinicaltrials.gov")]

In [ ]:
### Temp ###
# refs_nct = refs_nct.replace(to_replace=r'NCT00147641', value='NCT00159861', regex=True)

### Obsolete NCT ID, change it to the new one, but join back with the source.number

In [34]:
# Change search.date to datetime
refs_nct["search.date"] = [datetime.strptime(d, '%Y-%m-%d') for d in refs_nct["search.date"]]
# refs_nct["search.date"] = [datetime.strptime(d, '%m/%d/%Y') for d in refs_nct["search.date"]]

### Set time to 00:00?

In [8]:
def parsed_html(url, timeout=2000, verify=False, sleep_time=5):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.1.2222.33 Safari/537.36",
        "Accept-Encoding": "*",
        "Connection": "keep-alive"}
    
    # Download the context from the url
    page = requests.get(url, timeout=20, verify=False, headers = headers)
    
    # Check if HTTP status code is anything other than "ok"
    if page.status_code != 200:
        return None
    
    # Grab the raw HTML of the result
    raw_html = page.text
    
    # Use pd.read_html to convert it to a pd.dataframe
    df = pd.read_html(raw_html)[0]
    
    # We have added a default 1 sec sleep
    time.sleep(sleep_time)
    
    # Return the parsed results
    return df

In [73]:
# Loop over the subset to find the last change date for outcome measures on clinicatrials.gov
last_update = []

for i in refs_nct.url:
    
    # replace /show/ to /history/ in urls
    i = i.replace("show", "history")
    print(i)
    
    df_oc = parsed_html(i)
    
    #drop all na rows
    df_oc = df_oc.dropna(how = "all")
    
    # Find the most recent change for outcome measures
#     print(df[df["Changes"].str.contains("Outcome Measures")])
    df_last_oc = df_oc[df_oc["Changes"].str.contains("Outcome Measures")].tail(1)
    
    # if df does not contain outcome measures changes, print the message
    if df_last_oc.empty:
        print(f"{i} does not change outcomes before")
        last_update.append(np.nan)
    
    else:
        d_oc = df_last_oc["Submitted Date"].item()
    
        # Transform last date update to desired format (mm/dd/yyyy)
        update_time = datetime.strptime(d_oc, "%B %d, %Y")
        update_time_str = datetime.strftime(update_time, "%Y-%m-%d")
        print(update_time_str)

        # Save as a list
        last_update.append(update_time)

https://clinicaltrials.gov/ct2/history/NCT03317496
2022-05-26
https://clinicaltrials.gov/ct2/history/NCT03689855
2022-07-14


In [72]:
# Loop over the subset to find the last update date on clinicatrials.gov
last_post = []

for i in refs_nct.url:
    
    # replace /show/ to /archive/
    i = i.replace("show", "archive")
#     print(i)
    
    df = pd.read_html(i)[0]
    
# Get the last row value (assuming the last row is referred to last update posted)
#     print(df.iloc[-1])

    # Transform last date update to desired format (mm/dd/yyyy)
    ind = df[df[0] == "Last Update Posted:"].index
    d = str(df[1].iloc[ind[0]])
    post_time = datetime.strptime(d, "%B %d, %Y")
    post_time_str = datetime.strftime(post_time, "%Y-%m-%d")
    print(post_time_str)

    # Save as a list
    last_post.append(post_time)

2023-02-08
2022-08-11


In [25]:
dates = pd.DataFrame({"last_outcome_change": last_update,
                      "last_update_posted": last_post})
dates.to_csv("NCTs dates.csv", index = False, encoding = "utf-8")

In [35]:
# Comparing dates between last nct update and search date
refs_nct.insert(2, "last_outcome_change", last_update)
refs_nct.insert(3, "nct_update_needed", np.nan)
refs_nct.insert(4, "last_update_posted", last_post)
refs_nct.nct_update_needed = refs_nct["last_outcome_change"] > refs_nct["search.date"]
refs_nct["last_outcome_change"] = [datetime.strftime(d, "%Y-%m-%d") if pd.notnull(d) else "" for d in refs_nct["last_outcome_change"]]
refs_nct["last_update_posted"] = [datetime.strftime(d, "%Y-%m-%d") if pd.notnull(d) else "" for d in refs_nct["last_update_posted"]]

In [36]:
refs_merge = pd.merge(refs, refs_nct[["source.number", "last_outcome_change", "nct_update_needed", "last_update_posted"]], how = "left", on = "source.number")

In [37]:
# Convert nan to ""
refs_merge["last_outcome_change"] = refs_merge["last_outcome_change"].replace(np.nan, "")
refs_merge["search.date"] = refs_merge["search.date"].replace(np.nan, "")
refs_merge["last_update_posted"] = refs_merge["last_update_posted"].replace(np.nan, "")

In [38]:
# convert date to string
for i in range(len(refs_merge)):
    refs_merge["search.date"][i] = str("=\"") + refs_merge["search.date"][i] + str("\"")
    refs_merge["last_outcome_change"][i] = str("=\"") + refs_merge["last_outcome_change"][i] + str("\"")
    refs_merge["last_update_posted"][i] = str("=\"") + refs_merge["last_update_posted"][i] + str("\"")

# for i,j in zip(refs_merge["search.date"],refs_merge["last_nct_update"]):
#     i = str("=\"") + i + str("\"")
#     j = str("=\"") + j + str("\"")

In [39]:
refs_merge

,search,search.date,url,source.number,authors,publication.year,title,journal,volume,pages,...,control,n.study,n.arm,comment,location,e.copy,database,last_outcome_change,nct_update_needed,last_update_posted
0,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/33818208,33818208,"Bassanelli M, Ricciuti B, Giannarelli D, Cecer...",2022.0,Systemic effect of radiotherapy before or afte...,Tumori,108,250-257,...,NaN,NaN,NaN,NaN,NaN,NaN,no,"=""""",NaN,"="""""
1,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/34393061,34393061,"Remon J, Girard N, Novello S, de Castro J, Big...",2022.0,"PECATI: A Multicentric, Open-Label, Single-Arm...",Clin Lung Cancer,23,e243-e246,...,NaN,43,1,NaN,NaN,received,no,"=""""",NaN,"="""""
2,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/34456145,34456145,"Wong DJ, Bauer TM, Gordon MS, Bene-Tchaleu F, ...",2022.0,Safety and Clinical Activity of Atezolizumab P...,Clin Lung Cancer,23,273-281,...,no control,23,1,NaN,NaN,yes,yes,"=""""",NaN,"="""""
3,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/34470722,34470722,"Belluomini L, Dionisi V, Palmerio S, Vincenzi ...",2022.0,Study Design and Rationale for Espera Trial: A...,Clin Lung Cancer,23,e269-e272,...,pembrolizumab,NaN,2,NaN,NaN,received,no,"=""""",NaN,"="""""
4,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/34489161,34489161,"De Ruysscher D, Ramalingam S, Urbanic J, Gerbe...",2022.0,CheckMate 73L: A Phase 3 Study Comparing Nivol...,Clin Lung Cancer,23,e264-e268,...,NaN,888,3,NaN,NaN,yes,no,"=""""",NaN,"="""""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174,pubmed,"=""2022-09-08""",https://www.ncbi.nlm.nih.gov/pubmed/31435660,31435660,"Aguilar EJ, Ricciuti B, Gainor JF, Kehl KL, Kr...",2019.0,Outcomes to first-line pembrolizumab in patien...,Ann Oncol,30,1653-1659,...,no control,187,1,NaN,NaN,yes,yes,"=""""",NaN,"="""""
2175,clinicaltrials.gov,"=""2022-09-08""",https://clinicaltrials.gov/ct2/show/NCT02325739,388,Novartis,2020.0,"A Phase I/II, Multicenter, Open-label Study of...",clinicaltrials.gov,NaN,NaN,...,FGF401,172,7,NaN,NaN,yes,yes,"=""2020-11-21""",False,"=""2020-12-17"""
2176,clinicaltrials.gov,"=""2022-09-08""",https://clinicaltrials.gov/ct2/show/NCT02998528,389,BMS,2022.0,"Randomized, OpenLabel, Phase 3 Trial of Nivolu...",clinicaltrials.gov,NaN,NaN,...,chemotherapy,358,2,NaN,NaN,yes,yes,"=""2022-08-31""",False,"=""2023-02-10"""
2177,clinicaltrials.gov,"=""2022-09-08""",https://clinicaltrials.gov/ct2/show/NCT02901899,390,Northwestern University,2021.0,An Open Label Phase II Trial of Guadecitabine ...,clinicaltrials.gov,NaN,NaN,...,no control,45,1,NaN,NaN,yes,yes,"=""2021-04-16""",False,"=""2021-05-12"""


In [40]:
refs_merge.to_csv("NCTs update check.csv", index = False, encoding = "utf-8")